# KEPCO Test

In [4]:
import logging
logging.basicConfig(level=logging.INFO)

## 1 Environment

In [7]:
%reload_ext autoreload
%autoreload 2

from org.kepco import Kepco

import account
id=account.account_get("kepco", "id")
pw=account.account_get("kepco", "pw")
cert=account.account_get("kepco", "cert")
kepco = Kepco(id=id, pw=pw,cert_pw=cert)


INFO:WDM:====== WebDriver manager ======
INFO:WDM:Get LATEST edgedriver version for 120.0.2210 Edge
INFO:WDM:Driver [C:\Users\integ\.wdm\drivers\edgedriver\win64\120.0.2210\msedgedriver.exe] found in cache


## 2. Kepco Login

In [8]:
%autoreload 2
from org.kepco import Kepco

kepco.login()

True

## 2. Register

In [9]:
%autoreload 2

kepco.register("G022300192-00")


INFO:kepco:0. 모든 탭 닫기
INFO:kepco:No need to refresh
INFO:kepco:1. 공고번호 조회 tab 열기
ERROR:auto:Timeout! wait_until takes 3.075429677963257. timeout=3, interval=0.5, retry=7
INFO:kepco:1.2 search
INFO:kepco:1.3 validate


find registration panel
wait for a column


ERROR:auto:Timeout! wait_until takes 60.28618383407593. timeout=60, interval=0.5, retry=105
ERROR:kepco:Failed to get status column.
INFO:kepco:1.4 apply
ERROR:auto:Timeout! wait_until takes 60.401121854782104. timeout=60, interval=0.5, retry=113
ERROR:kepco:Failed to find a checkbox of the item
ERROR:kepco:Failed to apply


False

## 3. Participate

In [ ]:

%autoreload 2

import pyautogui
from org.kepco import *
import random
from selenium.webdriver.support import expected_conditions as when
from selenium.webdriver.support.ui import WebDriverWait



#--------------------------

# ID: Kepco - 입찰 - 검색결과 검증
# Description: 투찰진행상태를 확인한다. - 미제출
def can_participate(driver:WebDriver) -> bool :
    elem = auto_find_element(driver, By.XPATH, "//div[2]/div/div[2]/table/tbody/tr/td[2]/div")
    if elem.text == "미제출":
        return True
    else:
        return False


# ID: Kepco - 입찰 - 검색결과 검증
def validate_bid_search_result(driver:WebDriver, number):
    es = auto_find_all_elements(driver, By.XPATH, '//div[@class="x-grid-row-checker"]')
    if len(es) != 1:
        log().error(f"Failed to validate the result. Item should be only one. found={len(es)} ")
        return False

    # validate number
    e = auto_find_element(driver, By.XPATH, '//div[2]/table[1]/tbody/tr/td[5]/div[contains(@class,"x-grid-cell-inner")]')
    bid_num = e.text
    if number.find(bid_num) < 0:
        log().error(f"Failed to validate the result. Bid number missmatch. found={bid_num}, expected={number} ")
        return False

    return True
        



def participate(driver, number, cost):
    # close all tabs
    close_all_tab(driver)

    # open bid tab
    if not open_bid_tab(driver):
        log().error("Failed to open bind tab.")
        return False

    # ID: Kepco - 입찰 - 공고번호 검색
    # 공고번호 입력
    num_input = wait_clickable(driver, (By.XPATH, '//input[@title="공고번호"]'))
    num_input.send_keys(number)

    # 조회 버튼 클릭                                    
    # XXX: 가끔 click이 안되는 경우가 있다. 
    # auto_click(driver, By.XPATH, '//span[text()="조회"]')
    search_btn = wait_clickable(driver, (By.XPATH, '//span[text()="조회"]'))
    search_btn.click()
    
    # messagebox : 공고일자의 최대 검색일자는 6개월 입니다. "확인" 
    msgbox = kepco_get_messagebox(driver)    
    if msgbox:
        log().info(f"messagebox: text={msgbox.text}")
        okbtn = kepco_messagebox_get_button(msgbox, "확인")
        okbtn.click()

    # ID: Kepco - 입찰 - 검색결과 검증
    if not validate_bid_search_result(driver, number):
        log().error("Validation Failed.")
        return False

    # Description: 투찰진행상태 가져오기 
    if not can_participate(driver):
        log().info("Already Registered.")
        return True

    # ID: Kepco - 입찰 - 입찰참여 버튼
    # checkbox click
    log().info(f"Kepco - 입찰 - 입찰참여 버튼")
    auto_click(driver, By.XPATH, '//div[@class="x-grid-row-checker"]')
    # TODO: 안눌리는 경우가 생긴다.
    if not auto_click(driver, By.XPATH, '//span[text()="입찰참여"]'):
        log().error("Failed to click 입찰참여 button.")
        return False
                
    # messagebox - 입찰 창여 하시겠습니까?
    msgbox = kepco_get_messagebox(driver)
    log().info(f"messagebox: text={msgbox.text}")
    yesbtn = kepco_messagebox_get_button(msgbox, "예")
    yesbtn.click()

    # ID: Kepco - 입찰 - 입찰서 작성 - 지문인식 투찰 버튼 
    log().info(f"Kepco - 입찰 - 입찰서 작성 - 지문인식 투찰 버튼")
    auto_click(driver, By.XPATH, '//td[5]/div/img')


    # TODO: 시간이 오래된 경우 공인인증서 확인 창이 생성된다. 


    # messagebox - 지문인식투찰을 진행하시겠습니까?
    msgbox = kepco_get_messagebox(driver)
    log().info(f"messagebox: text={msgbox.text}")
    okbtn = kepco_messagebox_get_button(msgbox, "예")
    okbtn.click()

    # close all popup 
    log().info(f"Kepco - Common - 모든 팝업창 닫기")
    close_all_popup(driver, timeout = 15)


    # ID: Kepco - 입찰 - 추첨
    log().info(f"Kepco - Common - 추첨번호 선택")
    boxes = auto_find_all_elements(driver, By.XPATH, "//tr[3]/td/div/div/div/table/tbody/tr/td/a/span/span/span[2]")
    boxes = random.sample(boxes, k=4)
    for box in boxes:
        # 4개가 모두 click이 안되는 경우가 있다. 
        # 중복된 개체가 있는 것으로 보인다. 
        time.sleep(1)
        auto_click(driver, box)

    # ID: Kepco - 입찰 - 가격입력
    # ---------------------------
    cost_box = wait_element(driver, (By.XPATH, "//tbody/tr[4]/td/div[1]/div/div/table/tbody/tr[1]/td/div[1]"))

    # input layer
    input_layer = cost_box.find_element(By.XPATH, './/input')
    driver.execute_script(f'arguments[0].value = "{cost}"', input_layer)

    # focus
    text_layer = cost_box.find_element(By.XPATH, './/div[@class="x-form-field-inputcover-displayEl"]')
    text_layer.click()

    valid_box = wait_element(driver, (By.XPATH, "//tbody/tr[4]/td/div[1]/div/div/table/tbody/tr[2]/td/div[1]"))

    # input layer
    input_layer = valid_box.find_element(By.XPATH, './/input')
    driver.execute_script(f'arguments[0].value = "{cost}"', input_layer)

    # focus
    text_layer = valid_box.find_element(By.XPATH, './/div[@class="x-form-field-inputcover-displayEl"]')
    text_layer.click()

    # focus out to temp element.
    cost_box.click()

    # 입력값 확인 버튼 
    auto_click(driver, By.XPATH, '//span[text()="입력값확인"]')
    
    # ---------------------------
    # ID: Kepco - 입찰 - 제출
    # ---------------------------
    auto_click(driver, By.XPATH, '//span[text()="제출"]')
    msgbox = kepco_get_messagebox(driver)
    yesbtn = kepco_messagebox_get_button(msgbox, "예")
    yesbtn.click()

    #message box - 제출되었습니다. 
    msgbox = kepco_get_messagebox(driver)
    btn = kepco_messagebox_get_button(msgbox, "확인")
    btn.click()




In [ ]:

# number= 'G092200035-00'
# register(driver, number)

number = 'G052200237-00'
cost = "25436532"
participate(driver, number, cost)


# Kepco Close All Tab test


In [ ]:
from org.kepco import Kepco
import account 

id = account.account_get("kepco", "id")
pw = account.account_get("kepco", "pw")

kepco = Kepco(id, pw)



In [ ]:

# Refresh
kepco.driver.refresh()

# Accept alert
alert = kepco.driver.switch_to.alert
print(alert.text)
alert.accept()



In [ ]:
from auto import *

# validate login
# wait_element(kepco.driver, (By.XPATH, '//span[text()="로그아웃"]'), timeout=15)
wait_element(kepco.driver, (By.XPATH, '//span[text()="로그인"]/../../../a[contains(@style, display: none;)]'), timeout=15)

//a[contains(@style, "display: none;")]/span/span/span[2 and text()="로그인"]

